if you are training your model with missing data then you model will not able to train .


# Techniques :

* Remove Rows (Complete Case Analysis )
   * drawback -- loss data
   
* Imputation (fill them) 

   * Univariate (focus on single column)
   
       * numerical 
          * Mean / Median
          * Random 
          * End of Distribution
          
       * Categorical
          * Mode
          * Missing 
          
    * Mutivariate
        * Iterative Imputer 
        * KNN imputer
* others

##  Complete Case Analysis : means delete if any row has single missing value.
also called 'list-wise deletion' of cases,consists in discarding observation where values in any of the vaariables are missing..

Complete Case Anaylsis means literalyy analyzing only those observations for which there is information in all of the variables in the datasets

### Assumption for CCA 

* Missing Completly at Random 
    -- if data is missing at points poins(random rows) in datasets .
    
    
### Advantages /Disadvantages

#### Advantages 

* Easy to implement as no data manipulation required.

* Preserves Variables distribution( if data is MCAR ,then the distribution of vaiables of  the reduced datasets should match the distribution in the original dataset).

#### Disadvantages 
* it can exclude a large fraction of the original dataset(id missing data is abundant).

* Excluded observations could be informative for the analysis (if data is not missing at random).

* When using our models in production,the model will not know how to handle missing data.


### when to apply CCA ?


1. MCRA is there .
2. Missing data should be less than or equal to 5 % only.
3. ratio of CCA of categorial should be same



In [4]:
import pandas as pd

In [5]:
df=pd.read_csv("winemag-data-130k-v2.csv")

In [7]:
df.sample(4)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
92104,92104,France,"Fresh and fruity, this wine has a crisp, miner...",Les Vieilles Vignes,91,34.0,Burgundy,Pouilly-Fuissé,NaN,Roger Voss,@vossroger,Domaine Vincent Girardin 2013 Les Vieilles Vig...,Chardonnay,Domaine Vincent Girardin
51079,51079,Argentina,"Opens with thin fruit and alcoholic burn, and ...",NaN,83,13.0,Mendoza Province,Mendoza,NaN,Michael Schachner,@wineschach,Alberti 154 2008 Syrah (Mendoza),Syrah,Alberti 154
72664,72664,US,"Made with 75% Sangiovese, with the rest Cabern...",Patio,86,12.0,Washington,Yakima Valley,Columbia Valley,Sean P. Sullivan,@wawinereport,Chandler Reach 2012 Patio Red (Yakima Valley),Red Blend,Chandler Reach
14889,14889,US,"Barrel notes of vanilla, baking spice and dill...",Reserve Spice Cabinet Vineyard,87,42.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Mercer 2014 Reserve Spice Cabinet Vineyard Syr...,Syrah,Mercer


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [10]:
#there is so many missing values , lets see them
df.isnull().sum()

Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64

In [12]:
# to apply cca we must have less 5 % of data to be misisng only lets calculate percentage of data
df.isnull().mean()*100

Unnamed: 0                0.000000
country                   0.048472
description               0.000000
designation              28.825661
points                    0.000000
price                     6.921544
province                  0.048472
region_1                 16.347493
region_2                 61.136715
taster_name              20.192197
taster_twitter_handle    24.015357
title                     0.000000
variety                   0.000769
winery                    0.000000
dtype: float64

In [38]:
# so lets clean out columns where less than 5 is missing
col_to_remove=[i for i in df.columns if df[i].isnull().mean()*100 <5 and df[i].isnull().mean()>0]
col_to_remove


 

['country', 'province', 'variety']

lets see datatype of this columns

In [61]:
df[col_to_remove].sample(5)
df[col_to_remove].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   country   129908 non-null  object
 1   province  129908 non-null  object
 2   variety   129970 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB


In [62]:
df['variety'].value_counts()

Pinot Noir                    13272
Chardonnay                    11753
Cabernet Sauvignon             9472
Red Blend                      8946
Bordeaux-style Red Blend       6915
                              ...  
Cabernet Sauvignon-Barbera        1
Sauvignonasse                     1
Forcallà                          1
Meseguera                         1
Bobal-Cabernet Sauvignon          1
Name: variety, Length: 707, dtype: int64

In [63]:
len(df[col_to_remove].dropna())/len(df)

0.9995075824607028

In [64]:
new_df=df[col_to_remove].dropna()

In [65]:
df.shape,new_df.shape

((129971, 14), (129907, 3))

In [66]:
temp=pd.concat([
    df['country'].value_counts()/len(df),
    new_df['country'].value_counts()/len(new_df)
],axis=1)

In [67]:
temp

,country,country
US,0.419355,0.419562
France,0.169984,0.170068
Italy,0.150341,0.150415
Spain,0.051127,0.051152
Portugal,0.043787,0.043808
Chile,0.034408,0.034417
Argentina,0.029237,0.029252
Austria,0.025737,0.025749
Australia,0.017919,0.017928
Germany,0.016658,0.016666


##### we can see ratio of categorial data is maintained

In [59]:
temp2=pd.concat(
[
    df['province'].value_counts()/len(df),
    new_df['province'].value_counts() /len(new_df)
],axis=1)

In [60]:
temp2

,province,province
California,0.278885,0.279023
Washington,0.066469,0.066501
Bordeaux,0.045710,0.045733
Tuscany,0.045372,0.045394
Oregon,0.041340,0.041360
...,...,...
Messinia,0.000008,0.000008
Markopoulo,0.000008,0.000008
Paardeberg,0.000008,0.000008
Krania Olympus,0.000008,0.000008
